# Upload Data Sets (Tech_Hub)

## Data Sets

In [307]:
import pandas as pd

# Load the Excel files into Pandas DataFrames
df1 = pd.read_excel("Metropolitan_Data_on_Tech.xlsx")
df1.rename(columns={'Employment Employment_percent_relative_standard_error Hourly_mean_wage Annual_mean_wage Wage_percent_relative_standard_error Hourly_10th_percentile_wage Hourly_25th_percentile_wage Hourly_median_wage Hourly_75th_percentile_wage Hourly_90th_percentile_wage Annual_10th_percentile_wage_(2) Annual_25th_percentile_wage_(2) Annual_median_wage_(2) Annual_75th_percentile_wage_(2) Annual_90th_percentile_wage_(2) Employment_per_1,000_jobs Location_Quotient': 'Employment_Total'}, inplace=True)
df1.rename(columns={'Employment_per_Thousand_Jobs': 'Employment_per_1k_jobs'}, inplace=True)
df1

,Area_Name,Employment_Total,Employment_percent_relative_standard_error,Hourly_mean_wage,Annual_mean_wage,Wage_percent_relative_standard_error,Hourly_10th_percentile_wage,Hourly_25th_percentile_wage,Hourly_median_wage,Hourly_75th_percentile_wage,Hourly_90th_percentile_wage,Annual_10th_percentile_wage,Annual_25th_percentile_wage,Annual_median_wage,Annual_75th_percentile_wage,Annual_90th_percentile_wage,Employment_per_1k_jobs,Location_Quotient
0,"Abilene, TX (00-10180)",1040,3.5,34.49,71740,1.7,13,21.03,30.24,44.39,61.36,27040,43740,62900,92340,127620,14.248,0.42
1,"Aguadilla-Isabela, PR (00-10380)",550,21.5,,52300,2.2,10.02,14.01,22.83,33.99,40.51,20830,29140,47480,70700,84250,10.511,0.31
2,"Akron, OH (00-10420)",8200,1.5,43.59,90670,1.6,23.02,29.85,39.7,54.97,65.07,47890,62090,82570,114330,135350,25.566,0.75
3,Alaska nonmetropolitan area (02-00006),1070,4.2,42.13,87620,1.1,24.24,30.01,39.75,50.71,62.79,50420,62420,82680,105480,130600,10.356,0.3
4,"Albany, GA (00-10500)",910,3.1,40.31,83840,1.2,21.06,26.98,37.9,49.29,61.85,43800,56120,78820,102530,128660,15.293,0.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535,(8) Estimate not released.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
536,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
537,SOC code: Standard Occupational Classification...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
538,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [308]:
import pandas as pd

# Load the Excel files into Pandas DataFrames
df2 = pd.read_excel("Tech_Job_Gains.xlsx")

df2

,City,2018,2019,2020,2021,2022,2023,2024
0,Albuquerque,20534,21211,21437,21778,22898,23276,23596
1,Atlanta,209965,215036,214425,217589,227821,229584,235268
2,Austin,135358,144514,151594,157572,176248,180507,190186
3,Baltimore,100820,103080,103411,104078,104423,104838,106852
4,Boise,23384,23737,23506,24048,25387,25749,26592
5,Boston,262166,269471,266245,266780,270775,268934,274023
6,Buffalo,24205,23689,22550,22815,23981,24550,25255
7,Charleston,17654,17614,17890,17759,19162,19588,20468
8,Charlotte,79401,79943,83419,85263,89246,90859,93806
9,Chicago,245968,248120,241768,238965,246517,245800,247935


In [309]:
import pandas as pd

# Load the Excel files into Pandas DataFrames
df3 = pd.read_excel("Venture_Capitol.xlsx")

df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Location               150 non-null    object 
 1   Venture_Capital_Score  150 non-null    float64
dtypes: float64(1), object(1)
memory usage: 2.5+ KB


## Merging the Data Sets

In [310]:
import pandas as pd
import sqlite3

# Load the Excel files into Pandas DataFrames
df1 = pd.read_excel("Metropolitan_Data_on_Tech.xlsx")
df2 = pd.read_excel("Tech_Job_Gains.xlsx")


# Adjust specific city names to match between datasets
df2['City'] = df2['City'].replace('New York City', 'New York')
df1['Area_Name'] = df1['Area_Name'].replace('Washington-Arlington-Alexandria, DC-VA-MD-WV (00-47900)', 'Washington DC-Arlington-Alexandria, DC-VA-MD-WV (00-47900)')

# Create an in-memory SQLite database
conn = sqlite3.connect(":memory:")  # This avoids creating a physical file

# Load DataFrames into SQLite tables
df1.to_sql("Metro_Data", conn, if_exists="replace", index=False)
df2.to_sql("Tech_Gains", conn, if_exists="replace", index=False)

# SQL Query to merge the data based on matching names (Area_Name, City, Location)
query = """
SELECT DISTINCT *
FROM Metro_Data 
JOIN Tech_Gains
ON Metro_Data.Area_Name LIKE '%' || Tech_Gains.city || '%' 
AND Metro_Data.Area_Name NOT LIKE '%nonmetropolitan%'
GROUP BY Metro_Data.Area_Name
"""



# Load the merged result back into a Pandas DataFrame
merged_df = pd.read_sql(query, conn)

# Close the connection
conn.close()

# Save or analyze the merged data
merged_df.to_excel("merged_output.xlsx", index=False)  # Save as Excel
merged_df  # Display first few rows


,Area_Name,"Employment Employment_percent_relative_standard_error Hourly_mean_wage Annual_mean_wage Wage_percent_relative_standard_error Hourly_10th_percentile_wage Hourly_25th_percentile_wage Hourly_median_wage Hourly_75th_percentile_wage Hourly_90th_percentile_wage Annual_10th_percentile_wage_(2) Annual_25th_percentile_wage_(2) Annual_median_wage_(2) Annual_75th_percentile_wage_(2) Annual_90th_percentile_wage_(2) Employment_per_1,000_jobs Location_Quotient",Employment_percent_relative_standard_error,Hourly_mean_wage,Annual_mean_wage,Wage_percent_relative_standard_error,Hourly_10th_percentile_wage,Hourly_25th_percentile_wage,Hourly_median_wage,Hourly_75th_percentile_wage,...,Employment_per_Thousand_Jobs,Location_Quotient,City,2018,2019,2020,2021,2022,2023,2024
0,"Albuquerque, NM (00-10740)",11730,3.29999995231628,46.7,97140,2.40000009536743,20.9,30.12,42.53,61.91,...,29.6319999694824,0.870000004768372,Albuquerque,20534,21211,21437,21778,22898,23276,23596
1,"Atlanta-Sandy Springs-Roswell, GA (00-12060)",120180,1.5,52.5,109210,0.699999988079071,26.05,36.34,50.12,65.4,...,42.6520004272461,1.25,Atlanta,209965,215036,214425,217589,227821,229584,235268
2,"Austin-Round Rock, TX (00-12420)",81150,1.39999997615814,53.6,111490,2.59999990463257,24.43,35.07,50.64,67.85,...,65.6859970092773,1.92999994754791,Austin,135358,144514,151594,157572,176248,180507,190186
3,"Baltimore-Columbia-Towson, MD (00-12580)",64180,1.79999995231628,61.76,128450,3.40000009536743,30.93,42.44,60.23,77.72,...,49.2039985656738,1.44000005722046,Baltimore,100820,103080,103411,104078,104423,104838,106852
4,"Boise City, ID (00-14260)",10580,2.70000004768372,53.76,111810,7.80000019073486,22.64,30.15,41.56,60.36,...,28.4099998474121,0.829999983310699,Boise,23384,23737,23506,24048,25387,25749,26592
5,"Boston-Cambridge-Nashua, MA-NH (00-71650)",136640,1.29999995231628,61.92,128790,0.899999976158142,33.19,43.83,60.66,77.69,...,49.4720001220703,1.45000004768372,Boston,262166,269471,266245,266780,270775,268934,274023
6,"Buffalo-Cheektowaga-Niagara Falls, NY (00-15380)",11770,1.39999997615814,46.88,97510,0.800000011920929,23.48,31.71,43.26,59,...,22.3320007324219,0.660000026226044,Buffalo,24205,23689,22550,22815,23981,24550,25255
7,"Charleston, WV (00-16620)",2540,4,36.92,76790,1.70000004768372,19.45,24.23,32.45,45.94,...,24.7229995727539,0.730000019073486,Charleston,17654,17614,17890,17759,19162,19588,20468
8,"Charleston-North Charleston, SC (00-16700)",11770,2.79999995231628,49.1,102120,1.29999995231628,24.33,33.72,46.41,60.59,...,31.4039993286133,0.920000016689301,Charleston,17654,17614,17890,17759,19162,19588,20468
9,"Charlotte-Concord-Gastonia, NC-SC (00-16740)",60370,0.899999976158142,55.78,116020,0.5,28.47,39.38,55.06,68.69,...,46.0379981994629,1.35000002384186,Charlotte,79401,79943,83419,85263,89246,90859,93806


### 1. Add ***Tech Job Growth Column*** & ***Entry Level Wage Average (10th and 25th Percentile)***

In [311]:
import pandas as pd
Merged_Output = pd.read_excel("merged_output.xlsx")
Merged_Output.rename(columns={'Employment Employment_percent_relative_standard_error Hourly_mean_wage Annual_mean_wage Wage_percent_relative_standard_error Hourly_10th_percentile_wage Hourly_25th_percentile_wage Hourly_median_wage Hourly_75th_percentile_wage Hourly_90th_percentile_wage Annual_10th_percentile_wage_(2) Annual_25th_percentile_wage_(2) Annual_median_wage_(2) Annual_75th_percentile_wage_(2) Annual_90th_percentile_wage_(2) Employment_per_1,000_jobs Location_Quotient': 'Employment_Total'}, inplace=True)
Merged_Output.rename(columns={'Employment_per_Thousand_Jobs': 'Employment_per_1k_jobs'}, inplace=True)



final_value = Merged_Output['2023']
intial_value = Merged_Output['2018']
years = 6

Merged_Output['Average_7_Year_Growth_Rate'] = (((final_value / intial_value) ** (1 / years) - 1) * 100).round(3)
Merged_Output['Entry_Level_Tech_Wage'] = (Merged_Output['Annual_10th_percentile_wage'] + Merged_Output['Annual_25th_percentile_wage'])/2

# Optionally, you can save the result to a new Excel file
Merged_Output.to_excel("updated_output.xlsx", index=False)

df4 = pd.read_excel('updated_output.xlsx')
df4.head()


,Area_Name,Employment_Total,Employment_percent_relative_standard_error,Hourly_mean_wage,Annual_mean_wage,Wage_percent_relative_standard_error,Hourly_10th_percentile_wage,Hourly_25th_percentile_wage,Hourly_median_wage,Hourly_75th_percentile_wage,...,City,2018,2019,2020,2021,2022,2023,2024,Average_7_Year_Growth_Rate,Entry_Level_Tech_Wage
0,"Albuquerque, NM (00-10740)",11730,3.3,46.70,97140,2.4,20.90,30.12,42.53,61.91,...,Albuquerque,20534,21211,21437,21778,22898,23276,23596,2.111,53065
1,"Atlanta-Sandy Springs-Roswell, GA (00-12060)",120180,1.5,52.50,109210,0.7,26.05,36.34,50.12,65.40,...,Atlanta,209965,215036,214425,217589,227821,229584,235268,1.500,64890
2,"Austin-Round Rock, TX (00-12420)",81150,1.4,53.60,111490,2.6,24.43,35.07,50.64,67.85,...,Austin,135358,144514,151594,157572,176248,180507,190186,4.914,61875
3,"Baltimore-Columbia-Towson, MD (00-12580)",64180,1.8,61.76,128450,3.4,30.93,42.44,60.23,77.72,...,Baltimore,100820,103080,103411,104078,104423,104838,106852,0.653,76300
4,"Boise City, ID (00-14260)",10580,2.7,53.76,111810,7.8,22.64,30.15,41.56,60.36,...,Boise,23384,23737,23506,24048,25387,25749,26592,1.619,54905


### Final Clean

In [312]:
import pandas as pd

# Load the initial data from the Excel file
df5 = pd.read_excel("Final_Tech_Data.xlsx")

# Drop unnecessary columns from df5 as per your previous step

# Define the weights for each column in the calculation of the "Entry Level Tech Friendly Index"
weights = {
    'Employment_per_1k_jobs': 0.25,
    'Location_Quotient': 0.20,
    'Average_7_Year_Growth_Rate': 0.25,
    'Entry_Level_Tech_Wage': 0.25
}

# Calculate the "Entry Level Tech Friendly Index" by combining the weighted columns
df5['Tech_Friendly_Index'] = (
    df5['Employment_per_1k_jobs'] * weights['Employment_per_1k_jobs'] +
    df5['Location_Quotient'] * weights['Location_Quotient'] +
    df5['Average_7_Year_Growth_Rate'] * weights['Average_7_Year_Growth_Rate'] +
    df5['Entry_Level_Tech_Wage'] * weights['Entry_Level_Tech_Wage'] 
)

# Save the updated DataFrame with the new index back to an Excel file
df5.to_excel("Tech_Hub.xlsx", index=False)

# Optional: Print the updated DataFrame to check the new column
df5



,Area_Name,Employment_Total,Employment_per_1k_jobs,Location_Quotient,Average_7_Year_Growth_Rate,Entry_Level_Tech_Wage,Tech_Friendly_Index
0,"Albuquerque, NM (00-10740)",11730,29.632000,0.87,2.111,53065,13274.359750
1,"Atlanta-Sandy Springs-Roswell, GA (00-12060)",120180,42.652000,1.25,1.500,64890,16233.788000
2,"Austin-Round Rock, TX (00-12420)",81150,65.685997,1.93,4.914,61875,15486.785999
3,"Baltimore-Columbia-Towson, MD (00-12580)",64180,49.203999,1.44,0.653,76300,19087.752250
4,"Boise City, ID (00-14260)",10580,28.410000,0.83,1.619,54905,13733.923250
5,"Boston-Cambridge-Nashua, MA-NH (00-71650)",136640,49.472000,1.45,0.426,80095,20036.514500
6,"Buffalo-Cheektowaga-Niagara Falls, NY (00-15380)",11770,22.332001,0.66,0.236,57395,14354.524000
7,"Charleston, WV (00-16620)",2540,24.723000,0.73,1.748,45425,11363.013750
8,"Charleston-North Charleston, SC (00-16700)",11770,31.403999,0.92,1.748,60380,15103.472000
9,"Charlotte-Concord-Gastonia, NC-SC (00-16740)",60370,46.037998,1.35,2.272,70565,17653.597500


### Calculate "Tech Friendly Index"

# Confortable Living Index

In [313]:
import pandas as pd 

Entry_Level_Wage= pd.read_excel("Final_Tech_Data.xlsx")
Entry_Level_Wage.drop(['Employment_Total', 'Employment_per_1k_jobs', 'Location_Quotient', 'Average_7_Year_Growth_Rate'], axis=1, inplace=True)
Entry_Level_Wage.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Area_Name              56 non-null     object
 1   Entry_Level_Tech_Wage  56 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.0+ KB


In [314]:
import pandas as pd

COL = pd.read_excel("Forbes_Cost_of_Living.xlsx")
COL.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   City                51 non-null     object
 1   State               51 non-null     object
 2   Housing ($)         51 non-null     int64 
 3   Transportation ($)  51 non-null     int64 
 4   Food ($)            51 non-null     int64 
 5   Entertainment ($)   51 non-null     int64 
 6   Healthcare ($)      51 non-null     int64 
dtypes: int64(5), object(2)
memory usage: 2.9+ KB


In [315]:
import pandas as pd

TB = pd.read_excel("Tax_Burden.xlsx")
TB.head()

,State,Total Tax Burden (%),Property Tax Burden (%),Individual Income Tax Burden (%),Total Sales & Excise Tax Burden (%)
0,NY,0.1202,0.0436,0.0463,0.0303
1,HI,0.1180,0.0264,0.0382,0.0534
2,VT,0.1112,0.0485,0.0309,0.0318
3,ME,0.1074,0.0486,0.0259,0.0329
4,CA,0.1040,0.0273,0.0487,0.0280


### Calculating Entry_Level_Tech_Wage to Cost of Living ratios

In [316]:
import pandas as pd
import sqlite3

# Load the datasets
Entry_Level_Wage = pd.read_excel("Final_Tech_Data.xlsx")
COL = pd.read_excel("Forbes_Cost_of_Living.xlsx")

# Drop unnecessary columns from Entry_Level_Wage
Entry_Level_Wage.drop(['Employment_Total', 'Employment_per_1k_jobs', 'Location_Quotient', 'Average_7_Year_Growth_Rate'], axis=1, inplace=True)

# Standardize column names for easier SQL usage
COL.rename(columns={"Housing ($)": "Housing", "Transportation ($)": "Transportation", 
                    "Food ($)": "Food", "Entertainment ($)": "Entertainment", "Healthcare ($)": "Healthcare"}, inplace=True)

# Create an in-memory SQLite database
conn = sqlite3.connect(":memory:")

# Load the DataFrames into SQLite
Entry_Level_Wage.to_sql("Tech_Wages", conn, if_exists="replace", index=False)
COL.to_sql("Cost_Living", conn, if_exists="replace", index=False)

# SQL query to calculate the ratios
query = """
SELECT Cost_Living.City, Tech_Wages.Area_Name,
    (Tech_Wages.Entry_Level_Tech_Wage * 12 / Cost_Living.Housing) AS Wage_to_Housing_Ratio,
    (Tech_Wages.Entry_Level_Tech_Wage * 12 / Cost_Living.Transportation) AS Wage_to_Transportation_Ratio,
    (Tech_Wages.Entry_Level_Tech_Wage * 12 / Cost_Living.Food) AS Wage_to_Food_Ratio,
    (Tech_Wages.Entry_Level_Tech_Wage * 12 / Cost_Living.Entertainment) AS Wage_to_Entertainment_Ratio,
    (Tech_Wages.Entry_Level_Tech_Wage * 12 / Cost_Living.Healthcare) AS Wage_to_Healthcare_Ratio
FROM Tech_Wages
JOIN Cost_Living
ON Tech_Wages.Area_Name LIKE '%' || Cost_Living.City || '%' 
GROUP BY Tech_Wages.Area_Name
"""

# Execute query and store the result in a DataFrame
result_df = pd.read_sql(query, conn)

# Close the SQLite connection
conn.close()

# Display results
result_df.head()

# Optionally, save the result to an Excel file
result_df.to_excel("Tech_Wage_Ratios.xlsx", index=False)
result_df

,City,Area_Name,Wage_to_Housing_Ratio,Wage_to_Transportation_Ratio,Wage_to_Food_Ratio,Wage_to_Entertainment_Ratio,Wage_to_Healthcare_Ratio
0,Albuquerque,"Albuquerque, NM (00-10740)",636,1061,1591,3183,2122
1,Atlanta,"Atlanta-Sandy Springs-Roswell, GA (00-12060)",519,1112,1730,3114,2224
2,Austin,"Austin-Round Rock, TX (00-12420)",464,1142,1650,2970,2320
3,Baltimore,"Baltimore-Columbia-Towson, MD (00-12580)",763,1526,2289,4161,2953
4,Boise,"Boise City, ID (00-14260)",598,1197,1733,3294,2196
5,Boston,"Boston-Cambridge-Nashua, MA-NH (00-71650)",384,1201,1922,3203,2402
6,Buffalo,"Buffalo-Cheektowaga-Niagara Falls, NY (00-15380)",765,1252,1812,3826,2374
7,Charleston,"Charleston, WV (00-16620)",389,838,1297,2477,1651
8,Charleston,"Charleston-North Charleston, SC (00-16700)",517,1114,1725,3293,2195
9,Charlotte,"Charlotte-Concord-Gastonia, NC-SC (00-16740)",651,1302,2016,3681,2646


In [317]:
import pandas as pd
import sqlite3

# Load the datasets
Entry_Level_Wage = pd.read_excel("Final_Tech_Data.xlsx")
COL = pd.read_excel("Tax_Burden.xlsx")

# Drop unnecessary columns from Entry_Level_Wage
Entry_Level_Wage.drop(['Entry_Level_Tech_Wage', 'Employment_Total', 'Employment_per_1k_jobs', 'Location_Quotient', 'Average_7_Year_Growth_Rate'], axis=1, inplace=True)

# Standardize column names for easier SQL usage

# Create an in-memory SQLite database
conn = sqlite3.connect(":memory:")

# Load the DataFrames into SQLite
Entry_Level_Wage.to_sql("Tech_Wages", conn, if_exists="replace", index=False)
TB.to_sql("Tax_Burden", conn, if_exists="replace", index=False)

# SQL query to calculate the ratios
query = """
SELECT *
FROM Tech_Wages
JOIN Tax_Burden
ON Tech_Wages.Area_Name LIKE '%, %' || Tax_Burden.State || '% %'
GROUP BY Tech_Wages.Area_Name
"""

# Execute query and store the result in a DataFrame
result_df = pd.read_sql(query, conn)

# Close the SQLite connection
conn.close()

# Display results
result_df.head()

# Optionally, save the result to an Excel file
result_df.to_excel("Tax_Burden_by_City.xlsx", index=False)
result_df

,Area_Name,State,Total Tax Burden (%),Property Tax Burden (%),Individual Income Tax Burden (%),Total Sales & Excise Tax Burden (%)
0,"Albuquerque, NM (00-10740)",NM,0.0803,0.0186,0.0113,0.0504
1,"Atlanta-Sandy Springs-Roswell, GA (00-12060)",GA,0.0765,0.0250,0.0236,0.0279
2,"Austin-Round Rock, TX (00-12420)",TX,0.0756,0.0371,0.0000,0.0385
3,"Baltimore-Columbia-Towson, MD (00-12580)",MD,0.0926,0.0260,0.0400,0.0266
4,"Boise City, ID (00-14260)",ID,0.0786,0.0212,0.0247,0.0327
5,"Boston-Cambridge-Nashua, MA-NH (00-71650)",NH,0.0563,0.0451,0.0015,0.0097
6,"Buffalo-Cheektowaga-Niagara Falls, NY (00-15380)",NY,0.1202,0.0436,0.0463,0.0303
7,"Charleston, WV (00-16620)",WV,0.0845,0.0222,0.0261,0.0362
8,"Charleston-North Charleston, SC (00-16700)",SC,0.0750,0.0263,0.0200,0.0287
9,"Charlotte-Concord-Gastonia, NC-SC (00-16740)",NC,0.0789,0.0200,0.0268,0.0321


## Combine Both

In [318]:
import pandas as pd
import sqlite3

# Load the datasets
TWR = pd.read_excel("Tech_Wage_Ratios.xlsx")
TBC = pd.read_excel("Tax_Burden_by_City.xlsx")


# Create an in-memory SQLite database
conn = sqlite3.connect(":memory:")

# Load the DataFrames into SQLite
TWR.to_sql("Cost_by_City", conn, if_exists="replace", index=False)
TBC.to_sql("Tax_by_City", conn, if_exists="replace", index=False)

# SQL query to calculate the ratios
query = """
SELECT *
FROM Cost_by_City
JOIN Tax_by_City
ON Cost_by_City.Area_Name == Tax_by_City.Area_Name
"""

# Execute query and store the result in a DataFrame
result_df = pd.read_sql(query, conn)

# Close the SQLite connection
conn.close()

# Display results
result_df.head()

# Optionally, save the result to an Excel file
result_df = result_df.drop(columns=['City', 'State'])
result_df.to_excel("Final_Living_Data.xlsx", index=False)
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 11 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Area_Name                            55 non-null     object 
 1   Wage_to_Housing_Ratio                55 non-null     int64  
 2   Wage_to_Transportation_Ratio         55 non-null     int64  
 3   Wage_to_Food_Ratio                   55 non-null     int64  
 4   Wage_to_Entertainment_Ratio          55 non-null     int64  
 5   Wage_to_Healthcare_Ratio             55 non-null     int64  
 6   Area_Name                            55 non-null     object 
 7   Total Tax Burden (%)                 55 non-null     float64
 8   Property Tax Burden (%)              55 non-null     float64
 9   Individual Income Tax Burden (%)     55 non-null     float64
 10  Total Sales & Excise Tax Burden (%)  55 non-null     float64
dtypes: float64(4), int64(5), object(2)

### Calculating Financial_Comfort_Index

In [319]:
import pandas as pd

# Load the initial data from the Excel file
df = pd.read_excel("Final_Living_Data.xlsx")

# Drop unnecessary columns if needed (adjust according to your dataset)
# For example, you might drop the Area_Name if it's not needed for the calculations:
# df = df.drop(columns=['Area_Name'])

# Define the weights for each column in the calculation of the "Financial Comfort Index"
weights = {
    'Wage_to_Housing_Ratio': 0.232,
    'Wage_to_Transportation_Ratio': 0.186,
    'Wage_to_Food_Ratio': 0.161,
    'Wage_to_Healthcare_Ratio': 0.10,
    'Wage_to_Entertainment_Ratio': 0.05,
}

# Calculate the "Financial Comfort Index" by combining the weighted columns
df['Financial_Comfort_Index'] = (
    df['Wage_to_Housing_Ratio'] * weights['Wage_to_Housing_Ratio'] +
    df['Wage_to_Transportation_Ratio'] * weights['Wage_to_Transportation_Ratio'] +
    df['Wage_to_Food_Ratio'] * weights['Wage_to_Food_Ratio'] +
    df['Wage_to_Healthcare_Ratio'] * weights['Wage_to_Healthcare_Ratio'] +
    df['Wage_to_Entertainment_Ratio'] * weights['Wage_to_Entertainment_Ratio'] 
)

# Save the updated DataFrame with the new index back to an Excel file
df.to_excel("Financial_Hub.xlsx", index=False)

# Optional: Print the updated DataFrame to check the new column
df


,Area_Name,Wage_to_Housing_Ratio,Wage_to_Transportation_Ratio,Wage_to_Food_Ratio,Wage_to_Entertainment_Ratio,Wage_to_Healthcare_Ratio,Area_Name.1,Total Tax Burden (%),Property Tax Burden (%),Individual Income Tax Burden (%),Total Sales & Excise Tax Burden (%),Financial_Comfort_Index
0,"Albuquerque, NM (00-10740)",636,1061,1591,3183,2122,"Albuquerque, NM (00-10740)",0.0803,0.0186,0.0113,0.0504,972.399
1,"Atlanta-Sandy Springs-Roswell, GA (00-12060)",519,1112,1730,3114,2224,"Atlanta-Sandy Springs-Roswell, GA (00-12060)",0.0765,0.0250,0.0236,0.0279,983.870
2,"Austin-Round Rock, TX (00-12420)",464,1142,1650,2970,2320,"Austin-Round Rock, TX (00-12420)",0.0756,0.0371,0.0000,0.0385,966.210
3,"Baltimore-Columbia-Towson, MD (00-12580)",763,1526,2289,4161,2953,"Baltimore-Columbia-Towson, MD (00-12580)",0.0926,0.0260,0.0400,0.0266,1332.731
4,"Boise City, ID (00-14260)",598,1197,1733,3294,2196,"Boise City, ID (00-14260)",0.0786,0.0212,0.0247,0.0327,1024.691
5,"Boston-Cambridge-Nashua, MA-NH (00-71650)",384,1201,1922,3203,2402,"Boston-Cambridge-Nashua, MA-NH (00-71650)",0.0563,0.0451,0.0015,0.0097,1022.266
6,"Buffalo-Cheektowaga-Niagara Falls, NY (00-15380)",765,1252,1812,3826,2374,"Buffalo-Cheektowaga-Niagara Falls, NY (00-15380)",0.1202,0.0436,0.0463,0.0303,1130.784
7,"Charleston, WV (00-16620)",389,838,1297,2477,1651,"Charleston, WV (00-16620)",0.0845,0.0222,0.0261,0.0362,743.883
8,"Charleston-North Charleston, SC (00-16700)",517,1114,1725,3293,2195,"Charleston-North Charleston, SC (00-16700)",0.0750,0.0263,0.0200,0.0287,989.023
9,"Charlotte-Concord-Gastonia, NC-SC (00-16740)",651,1302,2016,3681,2646,"Charlotte-Concord-Gastonia, NC-SC (00-16740)",0.0789,0.0200,0.0268,0.0321,1166.430


---

# Machine Learning Begins


### 1. Loading Data Sets & Defining Targets

In [321]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix


# Load datasets
tech_data = pd.read_excel("Tech_Hub.xlsx")
affordability_data = pd.read_excel("Financial_Hub.xlsx")

# Merge datasets on 'Area_Name'
df = pd.merge(tech_data, affordability_data, on='Area_Name', how='inner')

# Define features (X) and targets (Y1: Tech Hub, Y2: Affordability)
X = df.drop(columns=['Area_Name'])
Y1 = df['Tech_Friendly_Index']  # Binary classification (Yes/No)
Y2 = df['Financial_Comfort_Index']  # Binary classification (Yes/No)

### 2. Split data into training and testing sets

In [326]:
# Standardize the features
df_clean = df.select_dtypes(include=['number'])
X = df_clean.drop(columns=['Tech_Friendly_Index', 'Financial_Comfort_Index'])  # Drop target columns
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data into training and testing sets
X_train, X_test, Y1_train, Y1_test = train_test_split(X_scaled, Y1, test_size=0.2, random_state=42)
X_train, X_test, Y2_train, Y2_test = train_test_split(X_scaled, Y2, test_size=0.2, random_state=42)

Linear Regression Performance:
Mean Squared Error: 0.0000
R^2 Score: 1.0000
----------------------------

Decision Tree Regressor Performance:
Mean Squared Error: 11267956.6645
R^2 Score: 0.0748
----------------------------

Random Forest Regressor Performance:
Mean Squared Error: 1663770.5893
R^2 Score: 0.8634
----------------------------

Linear Regression (Affordability) Performance:
Mean Squared Error: 249855309.4732
R^2 Score: -31604.7761
----------------------------

Decision Tree Regressor (Affordability) Performance:
Mean Squared Error: 289588447.6822
R^2 Score: -36630.8717
----------------------------

Random Forest Regressor (Affordability) Performance:
Mean Squared Error: 239239871.5545
R^2 Score: -30261.9623
----------------------------

